In [ ]:
!pip install corner emcee

In [ ]:
# import all packages
import corner,emcee
import pandas
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# read in some data we created for this example (.dat is a generic filename, it's just a text file)
data_filename='https://raw.githubusercontent.com/uofscphysics/STEM_Python_Course/Summer2020/02_Week2/Data/1D_intro_examples.dat'
example_data_1D = pandas.read_csv(data_filename,sep=',',header=0)#this file is separated by spaces and its first line contains the names of the columns (header) 
print(example_data_1D.head())

In [ ]:
#Let's plot the data, with error bars, that we read from file (See Day 2)
plt.errorbar(example_data_1D['x'], #x,y,and error are the column names
             example_data_1D['y'], 
             yerr=example_data_1D['error'],#yerr denotes an error in the y-direction for plotting
             fmt='.') #fmt is "format", saying that I want data marked by "points"
plt.xlabel('x') #set the x-axis label 
plt.ylabel('y') #set the y-axis label
plt.show()

In [ ]:
#The data were generated with a simple quadratic equation:
#ax^2+bx+c. 
def my_model(x,a,b,c): #We define the model described above
    return(a*x**2+b*x+c)

#### We can obtain best-fit parameters using a Markov Chain Monte Carlo algorithm (MCMC). The premise here is that we have a way of representing the "likelihood" of a given set of parameters being the "truth". This is done by having a function that compares your model (given a choice of parameters) to your data, returning a "likelihood" that increases for more likely choices of parameters (parameters that cause a better fit to the data). Once we have this function, the MCMC algorithm essentially just tries a set number of different combinations of your model parameters in a more efficient manner than simply random choices. Based on the final distribution of samples and resulting likelihoods, we obtain estimates of the best-fit parameters and posterior distributions. 

#### Now create such a likelihood function. Remember that before we used a $\chi^2$ function to estimate best-fit parameters. In that case, we wanted to minimize the $\chi^2$ statistic to find best-fit parameters. Now we want a likelihood function maximize instead of minimize, but the principle is the same (calculating how well a given choice of parameters reproduces the observed data given your model). Therefore, you can just use the opposite (negative) of your $\chi^2$ function as your likelihood function. For now, just leave your function as-is (we can add the negative in later).

In [ ]:

def chisq_likelihood(theta, args):
    #This function accepts an argument "theta", which is 
    #a list of model parameters a, b and c. It then calculates
    #a chi-square statistic that it returns, which compares
    #the observations, errors, and model provided in args.
    
    x, y, yerr = args #args is a list, so this is the same as x=args[0],y=args[1],yerr=args[2]. x,y, and yerr are numpy arrays, mod is a function.
    a,b,c = theta #theta is also a list, so it follows the same as args above
    model_observations = my_model(x,a,b,c) #mod (a model) is the 4th element of args, and it accepts x values, and the three model parameters a,b,c. Now model_observations contains the model values at every point in x (and is a numpy array)
    inv_sigma2 = 1./yerr**2 #The chi-square statistic contains an inverse-square error, which we calculate here
    chisquare = np.sum((y-model_observations)**2*inv_sigma2 )#calculate the chi-square statistic. 
    return chisquare




#### The prior function holds any information you know beforehand about the model parameters (e.g. bounds). The simplest form of prior just returns a probability of 1 if you are within specified bounds, or 0 if not. It is computationally more efficient to add numbers than multiply, and a $\chi^2$ is already the log of a probability, so we usually use a "log" prior instead of simply a prior so that it can be added directly to our likelihood. Your prior function should therefore return 0 if all parameters are within their bounds ($\ln(1)=0$), and `-np.inf` otherwise ($\ln(0)\rightarrow-\infty$)

In [ ]:
# Create a log-prior function. 
def lnprior(theta):#accepts the model parameters (theta)
    a,b,c = theta #set a,b,c (see above)
    
    #give the following bounds: a=(0,5),b=(-5,5),c=(0,10)
    if -20 < a < 20. and -20 < b < 20 and -20 < c < 20: #we are assuming a "uniform prior" on all parameters, which is the same as just giving each parameter bounds.
        return 0.0 #if you try parameters inside the bounds, return a probability of 1 (log(1)=0)
    return -np.inf #if you try parameters outside the bounds, return 0 (log(0)=-inf)



#### Okay that's it. Now we just need a function that combines the two functions written above, that should be maximized by the MCMC sampler. It should accept an array of parameters, as well as the data and uncertainty (`x,y,yerr`). So that the code below works, call your function `lnprob`.

In [ ]:
# This is now a log-likelihood function, which is commonly used.
def lnprob(theta, x, y, yerr): #accepts theta (the model parameters), and the same x,y,yerr, and mod from above
    lp = lnprior(theta) #get the probability from the prior function
    if not np.isfinite(lp): 
        return -np.inf #return a probability of negative infinity if the prior is negative infinity

    #the chisq_likelihood function returns a chi-square, 
    #which you want to be as small as possible. We are 
    #maximizing the likelihood here, so we take
    #the negative of the chi-square function. The 0.5 here comes from the definition
    #of a Gaussian distribution.
    
    #the total likelihood is the product of the prior and the likelihood 
    #(or the sum of the log-prior and log-likelihood)
    return lp - 0.5*chisq_likelihood(theta, [x, y, yerr]) 


#### Now we can finally set up an MCMC sampler. We need to define the number of dimensions (number of model parameters) and the number of "walkers" we want. Think of a large room, which contains all of the possible combinations of parameters for our model. Imagine you walk this room blindfolded, and every step you take you just ask someone else what the likelihood is that your current space is the "truth", before moving on. The more people walking around your room the better coverage you'll get, meaning you'll be able to try more independent regions of your parameter space. This is what `nwalkers` does below, it sets the number of blindfolded people walking around the room. The higher this number, the slower the code will run.

In [ ]:
#Set up the MCMC to sample the full parameter space
ndim, nwalkers = 3, 500 #number of parameters to fit (3); number of individual "walkers" that randomly sample the space. Choose any number, the higher the slower.
best_guess_parameters = np.array([1,1,1])

# This sets the starting position of each walker. It's just creating a list
# of length nwalkers, and number of columns equal to ndim. Each row contains
# a random location for a walker to start, based on a Gaussian
# centered on your best guess. You can tighten or loosen this constraint by
# changing the "starting_location_width" parameter
starting_location_width = 5
starting_positions = []
for i in range(nwalkers):
    starting_positions.append(best_guess_parameters + starting_location_width*np.random.randn(ndim))

#Let's just look at where walkers are starting in the a-b-c parameter space
#(Doesn't need to be included in your own code)
#Notice we get a random sampling near our best guess (blue lines)
corner.corner(starting_positions,
              labels=['a','b','c'],
              truths=best_guess_parameters,
              plot_contours=False,plot_density=False,plot_datapoints=True)

plt.show()

#### Okay this actually starts the sampler. The necessary parameters are the number of walkers, the number of dimensions, your function to maximize (the last one you wrote above), and an argument called `args` that is equal to a list or tuple of any arguments that need to be passed to your (log)-likelihood function apart from the array of parameter guesses.

In [ ]:
#set up the MCMC sampler
sampler = emcee.EnsembleSampler(nwalkers, 
                                ndim, 
                                lnprob, #the likelihood function to maximize 
                                args=(example_data_1D['x'], #the arguments passed to the likelihood functions (other than the model parameters)
                                      example_data_1D['y'], 
                                      example_data_1D['error']))
#run the MCMC with the starting positions we defined and 100 sampling points per walker.
#this is the equivalent of setting the number of steps each blindfolded person can 
#take in the room, using the example above.
n_samples_per_walker=100
output = sampler.run_mcmc(starting_positions, n_samples_per_walker) 

#### Okay let's see what we got! 

In [ ]:
#sampler.chain contains all of the samples from the MCMC.
print(sampler.chain.shape)
#It currently holds the samples separately for each walker.
#We don't care about what each walker does, so let's flatten it:
#The -1 here means we don't care how many rows it takes, 
#give us the same number of columns as we have parameters
samples = sampler.chain.reshape((-1, ndim)) 
print(samples.shape)
#So we tried 50000 total model realizations for 3 parameters

In [ ]:
#So what did we actually get from this? Let's use another
#python package to see the output of the MCMC sampling.
#The dashed line is one way of estimating the best-fit parameters
#from an MCMC sampler (the median of the samples). The blue line
#shows the true values.
fig = corner.corner(samples, #samples is defined above
                    labels=["$a$", "$b$","$c$"],#parameter labels
                    quantiles=[.5],
                    plot_contours=False,plot_density=False,
                    plot_datapoints=True)


plt.show()

In [ ]:
#But how close did we get to the true values?
#We can take the 50th percentile of the distributions you see above
#As our result. Incidentally we could calculate the uncertainty as 
#well, perhaps as the differences between the 84th and 50th for
#the upper uncertainty, and 50th and 16th for the lower uncertainty

#axis=0 means we want to calculate percentiles along columns, not rows
a_mcmc, b_mcmc,c_mcmc = np.percentile(samples, 50,axis=0)
a_mcmc_lower, b_mcmc_lower,c_mcmc_lower = np.percentile(samples, 16,axis=0)
a_mcmc_upper, b_mcmc_upper,c_mcmc_upper = np.percentile(samples, 84,axis=0)

print('a:',a_mcmc,'±(',a_mcmc_upper-a_mcmc,a_mcmc-a_mcmc_lower,')')
print('b:',a_mcmc,'±(',b_mcmc_upper-b_mcmc,b_mcmc-b_mcmc_lower,')')
print('c:',a_mcmc,'±(',c_mcmc_upper-c_mcmc,c_mcmc-c_mcmc_lower,')')


#### Plot the result. Show the data (with error bars), with your best-fit line going through. How does it look? You could compare it to your $\chi^2$ result if you want.

In [ ]:
#set up plotting the model over the data
plt.errorbar(example_data_1D['x'],
             example_data_1D['y'],
             yerr=example_data_1D['error'],
             fmt='.',
             label='Data')

plt.plot(example_data_1D['x'],
         my_model(example_data_1D['x'],a_mcmc,b_mcmc,c_mcmc),
         'r--',#make the line green and dashed
         label='MCMC Fit')




plt.xlabel('x')
plt.ylabel('y')
plt.legend()

plt.show()

#### Not bad! Play with the `nwalkers` and `n_samples_per_walker` to see what it does to computation time and accuracy. What extra information do we get from this approach, vs. the simple $\chi$-square minimization?